In [1]:
from epigenomic_dataset import load_epigenomes
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler

# The considered window size
window_size = 200
cell_line="HEK293"
regions="enhancers"

# Retrieving the input data
X, y = load_epigenomes(
    cell_line = cell_line,
    dataset = "fantom",
    regions = regions,
    window_size = window_size
)

y = y.values.ravel()

# Imputation of NaN Values
X[X.columns] = KNNImputer(n_neighbors=X.shape[0]//10).fit_transform(X)

# Robust normalization of the values
X[X.columns] = RobustScaler().fit_transform(X)

#X = X.values
shape=(200,4)
#type(X)
# Here one should feature selection. How can we do this?

In [2]:
from ucsc_genomes_downloader import Genome
assembly=r"C:\Users\matte\OneDrive\Desktop\Bioinformatica\genomes\hg19"
genome = Genome(assembly)

In [3]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [4]:
import pandas as pd
import numpy as np
from keras_bed_sequence import BedSequence

def to_bed(data:pd.DataFrame)->pd.DataFrame:
    """Return bed coordinates from given dataset."""
    return data.reset_index()[data.index.names]


def one_hot_encode(genome:Genome, data:pd.DataFrame, nucleotides:str="actg")->np.ndarray:
    return np.array(BedSequence(
        genome,
        bed=to_bed(data),
        nucleotides=nucleotides,
        batch_size=1
    ))

def flat_one_hot_encode(genome:Genome, data:pd.DataFrame, window_size:int, nucleotides:str="actg")->np.ndarray:
    return one_hot_encode(genome, data, nucleotides).reshape(-1, window_size*4).astype(int)

def to_dataframe(x:np.ndarray, window_size:int, nucleotides:str="actg")->pd.DataFrame:
    return pd.DataFrame(
        x,
        columns = [
            f"{i}{nucleotide}"
            for i in range(window_size)
            for nucleotide in nucleotides
        ]
    )

In [5]:
#sequences =  to_dataframe(
 #       flat_one_hot_encode(genome, X, window_size),
  #      window_size
  #  )


In [6]:
sequences=to_bed(X)
sequences

,chrom,chromStart,chromEnd,strand
0,chr1,839687,839887,.
1,chr1,840711,840911,.
2,chr1,845481,845681,.
3,chr1,855755,855955,.
4,chr1,856613,856813,.
...,...,...,...,...
65418,chrY,58986666,58986866,.
65419,chrY,58994460,58994660,.
65420,chrY,59019261,59019461,.
65421,chrY,59019818,59020018,.


In [7]:
from typing import Dict, Tuple, List

class Model:
    def __init__(self, name: str, model, **kwargs):
        self.name = name
        self.model = model
        self.kwargs = kwargs
        
    def __str__(self) -> str:
        return self.name
    
    def __repr__(self) -> str:
        return self.name
    
    def get_model(self) -> Tuple:
        return (self.model, self.kwargs)

In [8]:
from sklearn.tree import DecisionTreeClassifier

def get_decision_tree(name: str = 'DecisionTree', criterion: str = 'gini', max_depth: int = 50, 
                      random_state: int = 42, class_weight: str = 'balanced', **kwargs) -> Model:
    model = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        random_state=random_state,
        class_weight=class_weight
    )
    return Model(name, model, **kwargs)

Model.DecisionTree = get_decision_tree

In [9]:
Model.DecisionTree()

DecisionTree

In [10]:
from sklearn.ensemble import RandomForestClassifier
from multiprocessing import cpu_count


def get_random_forest(name: str = 'RandomForest', n_estimators: int = 500, criterion: str = 'gini', 
                      max_depth: int = 30, random_state: int = 42, 
                      class_weight: str = 'balanced', n_jobs: int = cpu_count, **kwargs):
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        random_state=random_state,
        class_weight=class_weight,
        n_jobs=n_jobs
    )
    return Model(name, model, **kwargs)

Model.RandomForest = get_random_forest

In [11]:
Model.RandomForest()

RandomForest

In [12]:
from tensorflow.keras.layers import Layer, Input, Flatten, Reshape, Dense, Conv2D, BatchNormalization, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC
from sklearn.model_selection import StratifiedShuffleSplit


In [13]:
from tensorflow.keras.layers import AlphaDropout, ThresholdedReLU

In [14]:
from tensorflow.keras.models import Sequential

def get_sequential(default_name: str = 'Sequential'):
    # no first and last layer
    def get_layers(*layers: Tuple[Layer]):
        def get_model(input_shape: Tuple[int], name: str = None, optimizer: str = 'nadam', 
                      loss: str = 'binary_crossentropy', metrics: List = None,
                      epochs: int = 1000, batch_size: int = 1024, 
                      validation_split: float = 0.1, shuffle: bool = True, verbose: bool = False, 
                      callbacks: List = None, **kwargs):
            name = name or default_name
            input_layer = Input(shape=input_shape)
            output_layer = Dense(1, activation="sigmoid")
            model = Sequential((input_layer,) + layers + (output_layer,), name)
            
            metrics = metrics or [
                "accuracy",
                AUC(curve="ROC", name="auroc"),
                AUC(curve="PR", name="auprc")
            ]
            model.compile(
                optimizer=optimizer,
                loss=loss,
                metrics=metrics
            )
            
            kwargs.update({
                'epochs': epochs,
                'batch_size': batch_size,
                'validation_split': validation_split,
                'shuffle': shuffle,
                'verbose': verbose,
                'callbacks': callbacks
            })
            model.summary()
            return Model(name, model, **kwargs)
        return get_model
    return get_layers

Model.Sequential = get_sequential()

In [15]:
Model.Sequential()((104,20))

Model: "Sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 104, 1)            21        
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________


Sequential

In [16]:
Model.Perceptron = get_sequential('Perceptron')()

In [17]:
Model.Perceptron((104,))

Model: "Perceptron"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 105       
Total params: 105
Trainable params: 105
Non-trainable params: 0
_________________________________________________________________


Perceptron

In [18]:
Model.MLP = get_sequential('MLP') 
Model.FFNN = get_sequential('FFNN')
Model.CNN = get_sequential('CNN')

In [19]:
#training 20 sec 
Model.MLP_Epi = Model.MLP(
    Dense(128, activation="relu"),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
)

In [20]:
#training 40 sec
Model.FFNN_Epi = Model.FFNN(
    Dense(256, activation="relu"),
    Dense(128),
    BatchNormalization(),
    Activation("relu"),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
)

In [21]:
#test con Alphadropout e ThresholdedReLU
Model.FFNN_Epi_2 = Model.FFNN(
    Dense(256, activation="relu"),
    Dense(128),
    BatchNormalization(),
    ThresholdedReLU(0.05),
    Dense(64, activation="relu"),
    AlphaDropout(0.3), #new
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
)

In [22]:
#test con
Model.FFNN_Epi_3 = Model.FFNN(
    Dense(256, activation="relu"),
    Dense(128, activation="relu"),
    BatchNormalization(),
    ThresholdedReLU(0.05),
    Dense(64, activation="relu"),
    Dense(64, activation="relu"),
    AlphaDropout(0.5), #new
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
)

In [23]:
Model.MLP_Seq = Model.MLP(
    Flatten(),
    Dense(64, activation="relu"),
    Dense(32, activation="relu")
)

In [24]:
Model.FFNN_Seq = Model.FFNN(
    Flatten(),
    Dense(128, activation="relu"),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
)

In [25]:
Model.CNN_Seq = Model.CNN(
    Reshape((200, 4, 1)),
    Conv2D(64, kernel_size=(10, 2), activation="relu"),
    Conv2D(64, kernel_size=(10, 2), activation="relu"),
    Dropout(0.3),
    Conv2D(32, kernel_size=(10, 2), strides=(2, 1), activation="relu"),
    Conv2D(32, kernel_size=(10, 1), activation="relu"),
    Conv2D(32, kernel_size=(10, 1), activation="relu"),
    Dropout(0.3),
    Flatten(),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
)

In [26]:
from ucsc_genomes_downloader import Genome
from keras_bed_sequence import BedSequence
from keras_mixed_sequence import MixedSequence
from tensorflow.keras.utils import Sequence


def get_holdout(train:np.ndarray, test:np.ndarray, bed:pd.DataFrame, labels:np.ndarray, genome:genome, batch_size=1024)->Tuple[Sequence, Sequence]:
    return (
        MixedSequence(
            x=BedSequence(genome, bed.iloc[train], batch_size=batch_size),
            y=labels[train],
            batch_size=batch_size
        ),
        MixedSequence(
            x= BedSequence(genome, bed.iloc[test], batch_size=batch_size),
            y=labels[test],
            batch_size=batch_size #batch_size maggiori, risultati + veloci
        )
    )

In [27]:
## from sklearn.model_selection import StratifiedShuffleSplit

splits = 2 #da testare un numero diverso di split
holdouts = StratifiedShuffleSplit(n_splits=splits, test_size=0.2, random_state=42)


In [28]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, average_precision_score
from sanitize_ml_labels import sanitize_ml_labels
import numpy as np

def report(y_true:np.ndarray, y_pred:np.ndarray)->np.ndarray:
    integer_metrics = accuracy_score, balanced_accuracy_score
    float_metrics = roc_auc_score, average_precision_score
    results1 = {
        sanitize_ml_labels(metric.__name__): metric(y_true, np.round(y_pred))
        for metric in integer_metrics
    }
    results2 = {
        sanitize_ml_labels(metric.__name__): metric(y_true, y_pred)
        for metric in float_metrics
    }
    return {
        **results1,
        **results2
    }

#si usano solo per categorizzazione, non hanno senso per regressione
# auroc: area sotto la curva di Receiver operating characteristic: analizza falsi positivi e negativi
# valore minimo è 0.5 (non impara niente)  valore massimo è 1 (modello perfetto)
# auprc: area sotto la curva di precision recall: va a computare per diversi treashold di precisioni diversi, la somma delle aree
#sottese sotto i diversi traingolini della precision-reacall. Un modello che non impara nulla ha AUPRC = 0 e un modello perfetto ha
# AUPRC = 1

In [29]:
def precomputed(results, model:str, holdout:int)->bool:
    df = pd.DataFrame(results)
    if df.empty:
        return False
    return (
        (df.model == model) &
        (df.holdout == holdout)
    ).any()

In [30]:
models=[]

models.append(Model.CNN_Seq(shape,name="CNN"))

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 200, 4, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 191, 3, 64)        1344      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 182, 2, 64)        81984     
_________________________________________________________________
dropout_3 (Dropout)          (None, 182, 2, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 87, 1, 32)         40992     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 78, 1, 32)         10272     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 69, 1, 32)         10272   

In [31]:
from tqdm.auto import tqdm
import pandas as pd

In [41]:
gpus = tf.config.experimental.list_physical_devices('XLA_GPU')
gpus

[PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [42]:
import json
import compress_json
#if os.path.exists("results1.json"):
 #   with open('results.json') as json_file:
#        results = json.load(json_file)
#else:
results = []
    
for i, (train_index, test_index) in tqdm(enumerate(holdouts.split(sequences, y)), total=splits, desc="Computing holdouts", dynamic_ncols=True):
    train, test = get_holdout(train_index, test_index, sequences, y, genome)
    for model in tqdm(models, total=len(models), desc="Training models", leave=False, dynamic_ncols=True):
        if precomputed(results, model.name, i):
            continue
        history = model.model.fit(
            train,
            steps_per_epoch=train.steps_per_epoch,
            validation_data=test,
            validation_steps=test.steps_per_epoch,
            epochs=1000,
            shuffle=True,
            verbose=False,
            callbacks=[
                EarlyStopping(monitor="val_loss", mode="min", patience=50),
            ]
        ).history
        scores = pd.DataFrame(history).iloc[-1].to_dict()
        results.append({
            "model":model.name,
            "run_type":"train",
            "holdout":i,
            **{
                key:value
                for key, value in scores.items()
                if not key.startswith("val_")
            }
        })
        results.append({
            "model":model.name,
            "run_type":"test",
            "holdout":i,
            **{
                key[4:]:value
                for key, value in scores.items()
                if key.startswith("val_")
            }
        })
        compress_json.local_dump(results,  f"results_{cell_line}_{regions}.json")

KeyboardInterrupt: 

In [ ]:
from barplots import barplots
df = pd.DataFrame(results)
#df = df.drop(columns=["holdout"])
df[:5]


barplots(
    df,
    groupby=["model", "run_type"],
    show_legend=False,
    height=5,
    orientation="horizontal"
)

from PIL import Image
from glob import glob

for x in glob("barplots/*.png"):
    display(Image.open(x))

In [ ]:
df
